In [2]:
import carla
import random

In [3]:
client = carla.Client('localhost', 2000)
world = client.get_world()

In [4]:
# Get the blueprint library and filter for the vehicle blueprints
vehicle_bp = world.get_blueprint_library().filter('*vehicle*')

Now we have the blueprints, we need to find some appropriate spots in the map to spawn our vehicles. Each CARLA map provides pre-defined spawn points spread evenly throughout the map on the roads for this purpose.

In [5]:
spawn_points = world.get_map().get_spawn_points()

# Spawn 50 vehicles randomly distributed throughout the map 
# for each spawn point, we choose a random vehicle from the blueprint library
# These vehicles will not move
for i in range(0,10):
    world.try_spawn_actor(random.choice(vehicle_bp), random.choice(spawn_points))

Now we should also add a vehicle that will be the centerpoint of our simulation. To train an autonomous agent we need to simulate a the vehicle that it the autonomous agent will control. In CARLA parlance, we often refer to this vehicle as the "Ego vehicle". 

In [6]:
ego_vehicle = world.spawn_actor(random.choice(vehicle_bp), random.choice(spawn_points)) # Everytime this runs, a new vehicle is created
spectator = world.get_spectator()
spectator.set_transform(ego_vehicle.get_transform())

## Add sensors

Modern autonomous vehicles understand and interpret their environment through an array of attached sensors. These sensors include things such as optical video cameras, optical flow cameras, LIDARs, RADARs and accelerometers. CARLA has models of numerous types of sensors built in to create training data for machine learning. The sensors can be attached to a vehicle, or they can be attached to a fixed point to model for example a CCTV camera.

Here we will attach a standard camera sensor to the ego vehicle to record some video data:

In [9]:
# Create a transform to place the camera on top of the vehicle
camera_init_trans = carla.Transform(carla.Location(z=1.5))

# We create the camera through a blueprint that defines its properties
camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')

# We spawn the camera and attach it to our ego vehicle
camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=ego_vehicle)

Once we have spawned the camera, we need to set it recording through the listen() method. The listen method takes as argument a callback that defines what to do with the data. You can either stream it to another program or save it to disk.

We will use a lambda function as a callback to save the data to disk:

In [10]:
# Start camera with PyGame callback
camera.listen(lambda image: image.save_to_disk('out/%06d.png' % image.frame))

In [ ]:
camera.stop() # Stop the camera
# camera.destroy() # Destroy the camera

This will save the data to the out/ folder as a series of PNG image files named according to the simulation frame number.

There are a multitude of different types of sensors to choose from. [Here](https://carla.readthedocs.io/en/latest/core_sensors/) you can delve deeper into the array of sensors available and how to use them.

### Sensors step-by-step

The class carla.Sensor defines a special type of actor able to measure and stream data.

- What is this data? It varies a lot depending on the type of sensor. All the types of data are inherited from the general carla.SensorData.
- When do they retrieve the data? Either on every simulation step or when a certain event is registered. Depends on the type of sensor.
- How do they retrieve the data? Every sensor has a listen() method to receive and manage the data.

Despite their differences, all the sensors are used in a similar way. 

carla.SensorData

Base class for all the objects containing data generated by a carla.Sensor. This objects should be the argument of the function said sensor is listening to, in order to work with them. Each of these sensors needs for a specific type of sensor data. Hereunder is a list of the sensors and their corresponding data.
- Cameras (RGB, depth and semantic segmentation): carla.Image.
- Collision detector: carla.CollisionEvent.
- GNSS sensor: carla.GnssMeasurement.
- IMU sensor: carla.IMUMeasurement.
- Lane invasion detector: carla.LaneInvasionEvent.
- LIDAR sensor: carla.LidarMeasurement.
- Obstacle detector: carla.ObstacleDetectionEvent.
- Radar sensor: carla.RadarMeasurement.
- RSS sensor: carla.RssResponse.
- Semantic LIDAR sensor: carla.SemanticLidarMeasurement.
- Cooperative awareness messages V2X sensor: carla.CAMEvent.
- Custom V2X messages V2X sensor: carla.CustomV2XEvent. 

### Setting

As with every other actor, find the blueprint and set specific attributes. This is essential when handling sensors. Their attributes will determine the results obtained. These are detailed in the sensors reference.

The following example sets a dashboard HD camera.

In [11]:
# Find the blueprint of the sensor.
blueprint = world.get_blueprint_library().find('sensor.camera.rgb')
# Modify the attributes of the blueprint to set image resolution and field of view.
blueprint.set_attribute('image_size_x', '1920')
blueprint.set_attribute('image_size_y', '1080')
blueprint.set_attribute('fov', '110')
# Set the time in seconds between sensor captures
blueprint.set_attribute('sensor_tick', '1.0')

### Spawning

attachment_to and attachment_type, are crucial. Sensors should be attached to a parent actor, usually a vehicle, to follow it around and gather the information. The attachment type will determine how its position is updated regarding said vehicle.

- Rigid attachment. Movement is strict regarding its parent location. This is the proper attachment to retrieve data from the simulation.
- SpringArm attachment. Movement is eased with little accelerations and decelerations. This attachment is only recommended to record videos from the simulation. The movement is smooth and "hops" are avoided when updating the cameras' positions.
- SpringArmGhost attachment. Like the previous one but without doing the collision test, so the camera or sensor could cross walls or other geometries.



When spawning with attachment, location must be relative to the parent actor. 

In [ ]:
transform = carla.Transform(carla.Location(x=0.8, z=1.7))
sensor = world.spawn_actor(blueprint, transform, attach_to=ego_vehicle)

: 

### Listening 

Every sensor has a listen() method. This is called every time the sensor retrieves data.

The argument callback is a lambda function. It describes what should the sensor do when data is retrieved. This must have the data retrieved as an argument. 

## Animate vehicles with traffic manager

Now we've added our traffic and ego vehicle to the simulation and started recording camera data, we now need to set the vehicles in motion using the Traffic manager. The Traffic manager is a component of CARLA that controls vehicles to autonomously move around the roads of the map within the simulation, following the road conventions and behaving like real road users.

We can find all the vehicles in the simulation using the world.get_actors() method, filtering for all the vehicles. We can then use the set_autopilot() method to hand over control of the vehicle to the Traffic Manager.



In [17]:
for vehicle in world.get_actors().filter('vehicle.*'):
    vehicle.set_autopilot(True)

Now your simulation is running, with numerous vehicles driving around the map and a camera recording data from one of those vehicles. This data can then be used to feed a machine learning algorithm for training an autonomous driving agent. The Traffic manager has many functions for customising traffic behaviour, learn more [here](https://carla.readthedocs.io/en/latest/tuto_G_traffic_manager/).

This is the most basic possible set up for a simulation, now you can go into further details deeper into documentation about the many extra sensors you can use to generate data, and the many other features of CARLA that can make your simulations more detailed and more realistic. 

## Assign a vehicle as the Ego Vehicle

The Ego Vehicle is an important concept to bear in mind when using CARLA. The Ego Vehicle refers to the vehicle that will be the focus of the simulation. In most CARLA use cases it's likely to be the vehicle to which you will attach your sensors and/or the vehicle that your autonomous driving machine learning stack will control. It is important because it serves as the basis for some simulation operations that help improve the efficiency of the simulation, like for example:

- Loading map tiles for Large Maps: Large Maps (like Town 12) are made up of tiles to that are only loaded when needed to improve CARLA performance. The position of the Ego Vehicle dictates which tiles are used. Only the tiles nearest the Ego Vehicle will be loaded.

- Hybrid Physics Mode: if your simulation contains a lot of vehicles controlled by the Traffic Manager, calculating physics for all of these vehicles is very computationally expensive. The Hybrid Physics Mode enables physics calculation to be limited to the vehicles in the vicinity of the Ego Vehicle, hence saving computing resources.

To define the Ego Vehicle, you should set the role_name attribute of the vehicle carla.Actor object's blueprint when you are spawning your Ego Vehicle:

In [18]:
ego_bp = world.get_blueprint_library().find('vehicle.lincoln.mkz_2020')
ego_bp.set_attribute('role_name', 'hero')
ego_vehicle = world.spawn_actor(ego_bp, random.choice(spawn_points))

In [19]:
spectator.set_transform(ego_vehicle.get_transform())

## Choose your map

CARLA comes loaded with several pre-made maps focused on providing a diversity of features. The maps present a range of environments such as urban, rural and residential. There are also differing architectural styles and also a multitude of different road layouts from unmarked rural roads to multi-lane highways. Browse the map guides in the [catalogue](https://carla.readthedocs.io/en/latest/catalogue/) or in the table below.

In [ ]:
client.get_available_maps() # Avaliable maps
client.load_world('Town01') # Load a new map, TENDS TO CRASH

['/Game/Carla/Maps/Town04',
 '/Game/Carla/Maps/Town01',
 '/Game/Carla/Maps/Town02',
 '/Game/Carla/Maps/Town01_Opt',
 '/Game/Carla/Maps/Town03_Opt',
 '/Game/Carla/Maps/Town10HD_Opt',
 '/Game/Carla/Maps/Town07',
 '/Game/Carla/Maps/Town05',
 '/Game/Carla/Maps/Town02_Opt',
 '/Game/Carla/Maps/Town10HD',
 '/Game/Carla/Maps/Town04_Opt',
 '/Game/Carla/Maps/Town05_Opt',
 '/Game/Carla/Maps/Town03',
 '/Game/Carla/Maps/Town06',
 '/Game/Carla/Maps/Town11/Town11',
 '/Game/Carla/Maps/Town13/Town13',
 '/Game/Carla/Maps/Town15/Town15',
 '/Game/Carla/Maps/Town12/Town12']

## Choose your vehicle

In [ ]:
for bp in world.get_blueprint_library().filter('vehicle'):
    print(bp.id)

vehicle.audi.a2
vehicle.citroen.c3
vehicle.chevrolet.impala
vehicle.dodge.charger_police_2020
vehicle.micro.microlino
vehicle.dodge.charger_police
vehicle.audi.tt
vehicle.jeep.wrangler_rubicon
vehicle.mercedes.coupe
vehicle.yamaha.yzf
vehicle.mercedes.coupe_2020
vehicle.harley-davidson.low_rider
vehicle.dodge.charger_2020
vehicle.ford.ambulance
vehicle.lincoln.mkz_2020
vehicle.mini.cooper_s_2021
vehicle.ford.crown
vehicle.toyota.prius
vehicle.carlamotors.european_hgv
vehicle.carlamotors.carlacola
vehicle.vespa.zx125
vehicle.nissan.patrol_2021
vehicle.mercedes.sprinter
vehicle.audi.etron
vehicle.seat.leon
vehicle.volkswagen.t2_2021
vehicle.tesla.cybertruck
vehicle.lincoln.mkz_2017
vehicle.carlamotors.firetruck
vehicle.ford.mustang
vehicle.volkswagen.t2
vehicle.mitsubishi.fusorosa
vehicle.tesla.model3
vehicle.diamondback.century
vehicle.gazelle.omafiets
vehicle.bmw.grandtourer
vehicle.bh.crossbike
vehicle.kawasaki.ninja
vehicle.nissan.patrol
vehicle.nissan.micra
vehicle.mini.cooper_s


: 